In [ ]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2021-03-03 13:11:19--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2021-03-03 13:11:19--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc974ffbc0662204180a4c7c0c53.dl.dropboxusercontent.com/zip_download_get/AtZAho1GtteCPOECr04VPLf7ZaDVUaJXVQzR8RoSJSYfSrLAH8LpsPow94Botu9zRHaVb0JeEWCg5X5XSrnzBPhV6VbZZjqZj2Zcedu2XNeMOg [following]
--2021-03-03 13:11:19--  https://uc974ffbc0662204180a4c7c0c53.dl.dropboxusercontent.com/zip_download_get/AtZAho1GtteCPOECr04VPLf7ZaDVUaJXVQzR8RoSJSYfSrLAH8LpsPow94Botu9zRHaVb0JeEWCg5X5XS

In [ ]:
!unzip /content/AAAw1_JIzpuVvwteJCma0xMla?dl=0


Archive:  /content/AAAw1_JIzpuVvwteJCma0xMla?dl=0
mapname:  conversion of  failed
 extracting: train.csv               

1 archive had fatal errors.


In [ ]:
import pandas as pd
data = pd.read_csv(r"/content/train.csv")

In [ ]:
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [ ]:
data.drop(["qid"],axis=1,inplace=True)

In [ ]:
data.head()

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2021-03-03 13:11:24--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2021-03-03 13:11:24--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2021-03-03 13:11:25--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [ ]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [ ]:
!rm /content/glove.42B.300d.zip

In [ ]:
import numpy as np

In [ ]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()
# this again will take good amount of time

In [ ]:
from sklearn.model_selection import train_test_split
data_train,data_test=train_test_split(data,test_size=0.2)

In [ ]:
x_train=data_train['question_text']
y_train=data_train['target']
x_test=data_test['question_text']
y_test=data_test['target']

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM,Activation,Dense,Input,Embedding,Dropout
from tensorflow.keras.models import Model

In [ ]:
from nltk import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sent_lens=[len(word_tokenize(x)) for x in x_train]

In [ ]:
np.percentile(sent_lens,95)

30.0

In [ ]:
max_len=30

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [ ]:
embedding_matrix=np.zeros((vocab_size+1,300))
words_not_available=0
for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector
    else:
      words_not_available+=1

In [ ]:
words_not_available

62183

In [ ]:
vocab_size

196171

In [ ]:

inputs=Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer=LSTM(50)(embed)
dense1=Dense(10,activation='relu')(lstm_layer)
drop=Dropout(0.2)(dense1)
final_layer=Dense(1,activation='sigmoid')(drop)

model=Model(inputs=inputs,outputs=final_layer)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 300)           58851600  
_________________________________________________________________
lstm (LSTM)                  (None, 50)                70200     
_________________________________________________________________
dense (Dense)                (None, 10)                510       
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 58,922,321
Trainable params: 70,721
Non-trainable params: 58,851,600
____________________________________________

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from sklearn.utils import class_weight
cw = class_weight.compute_class_weight('balanced', 
                                                 np.unique(y_train), 
                                                    y_train)

In [ ]:
class_weight_dict = dict(enumerate(cw))
class_weight_dict

{0: 0.5329704016697645, 1: 8.082558517303795}

In [ ]:
model.fit(seq_train_matrix,y_train.values,validation_data=(seq_test_matrix,y_test.values),epochs=40,class_weight=class_weight_dict,
          batch_size=30000)

Epoch 1/40
35/35 [==============================] - 29s 622ms/step - loss: 0.5956 - accuracy: 0.5336 - val_loss: 0.3236 - val_accuracy: 0.8695
Epoch 2/40
35/35 [==============================] - 20s 586ms/step - loss: 0.3516 - accuracy: 0.8554 - val_loss: 0.3143 - val_accuracy: 0.8699
Epoch 3/40
35/35 [==============================] - 20s 589ms/step - loss: 0.3179 - accuracy: 0.8645 - val_loss: 0.3045 - val_accuracy: 0.8699
Epoch 4/40
35/35 [==============================] - 21s 589ms/step - loss: 0.3065 - accuracy: 0.8680 - val_loss: 0.3064 - val_accuracy: 0.8680
Epoch 5/40
35/35 [==============================] - 21s 592ms/step - loss: 0.2982 - accuracy: 0.8705 - val_loss: 0.2655 - val_accuracy: 0.8892
Epoch 6/40
35/35 [==============================] - 21s 596ms/step - loss: 0.2920 - accuracy: 0.8741 - val_loss: 0.2748 - val_accuracy: 0.8848
Epoch 7/40
35/35 [==============================] - 21s 599ms/step - loss: 0.2864 - accuracy: 0.8774 - val_loss: 0.2794 - val_accuracy: 0.8801

In [ ]:
p=model.predict(seq_test_matrix)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,p)